<a href="https://colab.research.google.com/github/SvA1/Compling/blob/master/HW4_few_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install deeppavlov
!pip install tensorflow==1.15.0
!apt-get --yes install git
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv

In [0]:
import pandas as pd
import json

In [0]:
from deeppavlov import configs, build_model, train_model

In [0]:
with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as file:
    ner_config = json.load(file)

In [0]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [0]:
ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

In [0]:
ner_model = train_model(ner_config, download=True)

In [15]:
ner_model(['asassins creed', 'FIfa'])

[[['asassins', 'creed'], ['FIfa']], [['B-GAME', 'O'], ['B-GAME']]]

In [0]:
marked = []

for text in data.text.values[:1000]:
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [16]:
marked [:20]

[[('Battlefield', 'B-GAME'),
  ('4', 'I-GAME'),
  (',', 'O'),
  ('Granturismo', 'B-GAME'),
  ('3', 'I-GAME'),
  ('и', 'O'),
  ('Звезды', 'O'),
  ('Playstation', 'B-GAME'),
  ('Битва', 'O'),
  ('сильнейших', 'O'),
  ('.', 'O'),
  ('За', 'O'),
  ('все', 'O'),
  ('1500', 'O'),
  ('руб', 'O'),
  ('.', 'O')],
 [('ИГРЫ', 'B-GAME'),
  ('ПО', 'O'),
  ('САМЫМ', 'B-GAME'),
  ('ДОСТУПНЫМ', 'B-GAME'),
  ('ЦЕНАМ', 'B-GAME'),
  ('\xa0', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('У', 'O'),
  ('нас', 'O'),
  ('всегда', 'O'),
  ('широчайший', 'O'),
  ('ассортимент', 'O'),
  ('лицензионных', 'O'),
  ('игр', 'O'),
  ('на', 'O'),
  ('PS4', 'B-GAME'),
  ('\\', 'O'),
  ('PS3', 'B-GAME'),
  ('\xa0', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('ОБМЕН', 'B-GAME'),
  ('ПРОДАЖА', 'B-GAME'),
  ('новых', 'O'),
  ('и', 'O'),
  ('б', 'O'),
  ('\\', 'O'),
  ('у', 'O'),
  ('игр', 'O'),
  ('для', 'O'),
  ('PS4', 'B-GAME'),
  ('PS3', 'B-GAME'),
  ('\xa0', 'O'),
  ('/', 'O'

# Анализ

В процессе разметки столкнулась с такой проблемой: после названия игры указывалось несколько ее частей или названия разных частей. В таком  случае не ясно было, надо ли разметить разные части игры просто как I-GAME или разметить только часть(или название части), которое идет непосредственно после названия игры.
 Xbox и ps4 размечаются как игры, скорее всего так как включила малое количество размеченных текстов,в которых они встречаются, в train.
 Опять же так как в train была часть предложений вида
` Fifa - 100p.`
то в размеченных данных много игр, записанных в этом формате разметились, но вместе с ними разметились и, например, "джойстик"
